In [1]:
import uuid
from functools import partial

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)


def is_subword(text, tokenized, tokenizer, index):
    word = tokenizer.convert_ids_to_tokens(tokenized["input_ids"][index])
    start_ind, end_ind = tokenized["offset_mapping"][index]
    word_ref = text[start_ind:end_ind]
    is_subword = len(word) != len(word_ref)
    return is_subword


def tokenize(example, labels2int, tokenizer, iob=True, ignore_subwords=True):

    text, labels = example["source_text"], example["privacy_mask"]

    i = 0
    token_labels = []

    tokenized = tokenizer(text, return_offsets_mapping=True, return_special_tokens_mask=True)
    start_token_to_label = {
        tokenized.char_to_token(label["start"]): (label["start"], label["end"], label["label"]) for label in labels
    }
    while i < len(tokenized["input_ids"]):
        if tokenized["special_tokens_mask"][i] == 1:
            token_labels.append(-100)
            i += 1
        elif i not in start_token_to_label:
            if ignore_subwords and is_subword(text, tokenized, tokenizer, i):
                token_labels.append(-100)
            else:
                token_labels.append(labels2int["O"])
            i += 1
        else:
            start, end, label = start_token_to_label[i]
            start_token = tokenized.char_to_token(start)
            assert start_token == i
            j = start_token
            while j < (len(tokenized["input_ids"]) - 1) and tokenized.token_to_chars(j).start < end:
                if j == start_token:
                    if iob:
                        token_labels.append(labels2int["B-" + label])
                    else:
                        token_labels.append(labels2int[label])
                elif ignore_subwords and is_subword(text, tokenized, tokenizer, j):
                    token_labels.append(-100)
                else:
                    if iob:
                        token_labels.append(labels2int["I-" + label])
                    else:
                        token_labels.append(labels2int[label])

                j += 1
            i = j
    tokenized["labels"] = token_labels
    return tokenized


def compute_metrics(eval_pred, label_list, seqeval_metric):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval_metric.compute(predictions=true_predictions, references=true_labels)
    results_flat = {f"{k}_f1": v["f1"] for k, v in results.items() if isinstance(v, dict)}
    results_flat.update(
        {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }
    )
    return results_flat


/usr/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-21 13:57:27.829353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-21 13:57:27.913210: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-21 13:57:27.926945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-21 13:57:28.019258: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use av

In [2]:
labels = [
    "BOD",
    "BUILDING",
    "CARDISSUER",
    "CITY",
    "COUNTRY",
    "DATE",
    "DRIVERLICENSE",
    "EMAIL",
    "GEOCOORD",
    "GIVENNAME1",
    "GIVENNAME2",
    "IDCARD",
    "IP",
    "LASTNAME1",
    "LASTNAME2",
    "LASTNAME3",
    "PASS",
    "PASSPORT",
    "POSTCODE",
    "SECADDRESS",
    "SEX",
    "SOCIALNUMBER",
    "STATE",
    "STREET",
    "TEL",
    "TIME",
    "TITLE",
    "USERNAME",
]

labels = [f"I-{label}" for label in labels] + [f"B-{label}" for label in labels] + ["O"]
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}
pretrained_name = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_name)
model = AutoModelForTokenClassification.from_pretrained(pretrained_name, num_labels=len(labels), id2label=id2label)

ds = load_dataset("ai4privacy/pii-masking-300k")
ds = ds.filter(lambda x: x["language"] == "English", num_proc=4)
ds = ds.map(
    partial(tokenize, labels2int=label2id, tokenizer=tokenizer, iob=True, ignore_subwords=True),
    batched=False,
    remove_columns=[
        "source_text",
        "target_text",
        "privacy_mask",
        "span_labels",
        "mbert_text_tokens",
        "mbert_bio_labels",
        "id",
        "language",
        "set",
    ],
    num_proc=8,
).remove_columns(["offset_mapping"])

training_arguments = TrainingArguments(
    output_dir="output",
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    overwrite_output_dir=True,
    warmup_ratio=0.2,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",
    warmup_steps=3000,
    metric_for_best_model="f1",
    greater_is_better=True,
    torch_compile=False,
)
trainer = Trainer(
    model,
    training_arguments,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    data_collator=DataCollatorForTokenClassification(tokenizer),
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, label_list=labels, seqeval_metric=evaluate.load("seqeval")),
)


/home/arnesh/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the latest cached version of the dataset since ai4privacy/pii-masking-300k couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/arnesh/.cache/huggingface/datasets/ai4privacy___pii-masking-300k/default/0.0.0/3b99ae5980049a0dff660f424916afa995

In [3]:
trainer.train()


  1%|          | 500/46750 [02:25<3:31:26,  3.65it/s]

{'loss': 1.8906, 'grad_norm': 1.5857051610946655, 'learning_rate': 3.3333333333333333e-06, 'epoch': 0.53}


  2%|▏         | 935/46750 [04:30<3:11:14,  3.99it/s]/home/arnesh/.local/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
                                                     
  2%|▏         | 935/46750 [05:04<3:11:14,  3.99it/s]

{'eval_loss': 0.15687373280525208, 'eval_BOD_f1': 0.8350714561606798, 'eval_BUILDING_f1': 0.8203125, 'eval_CARDISSUER_f1': 0.0, 'eval_CITY_f1': 0.475298126064736, 'eval_COUNTRY_f1': 0.7111431662459431, 'eval_DATE_f1': 0.674775557486244, 'eval_DRIVERLICENSE_f1': 0.5127231295100645, 'eval_EMAIL_f1': 0.8675543097407148, 'eval_GEOCOORD_f1': 0.2532188841201717, 'eval_GIVENNAME1_f1': 0.42192853646598144, 'eval_GIVENNAME2_f1': 0.0, 'eval_IDCARD_f1': 0.3282051282051282, 'eval_IP_f1': 0.8661864221785637, 'eval_LASTNAME1_f1': 0.4767580452920143, 'eval_LASTNAME2_f1': 0.0, 'eval_LASTNAME3_f1': 0.0, 'eval_PASS_f1': 0.5112866817155756, 'eval_PASSPORT_f1': 0.5995800839832034, 'eval_POSTCODE_f1': 0.626850168787328, 'eval_SECADDRESS_f1': 0.14598540145985403, 'eval_SEX_f1': 0.7146829810901001, 'eval_SOCIALNUMBER_f1': 0.5713782164258018, 'eval_STATE_f1': 0.7562836624775583, 'eval_STREET_f1': 0.4586220953933958, 'eval_TEL_f1': 0.6178059666431759, 'eval_TIME_f1': 0.8856878551605656, 'eval_TITLE_f1': 0.5777

  2%|▏         | 1000/46750 [05:24<3:43:52,  3.41it/s] 

...

                                                       
100%|██████████| 46750/46750 [4:14:37<00:00,  3.77it/s]

{'eval_loss': 0.07163183391094208, 'eval_BOD_f1': 0.9608096468561584, 'eval_BUILDING_f1': 0.9815856777493606, 'eval_CARDISSUER_f1': 1.0, 'eval_CITY_f1': 0.9649595687331537, 'eval_COUNTRY_f1': 0.9744721084147495, 'eval_DATE_f1': 0.9173722627737226, 'eval_DRIVERLICENSE_f1': 0.9400864731315627, 'eval_EMAIL_f1': 0.9857495724871745, 'eval_GEOCOORD_f1': 0.9836829836829837, 'eval_GIVENNAME1_f1': 0.8495192307692307, 'eval_GIVENNAME2_f1': 0.7303058387395738, 'eval_IDCARD_f1': 0.9345971563981043, 'eval_IP_f1': 0.9868208092485549, 'eval_LASTNAME1_f1': 0.8009950248756219, 'eval_LASTNAME2_f1': 0.6885496183206107, 'eval_LASTNAME3_f1': 0.6611111111111112, 'eval_PASS_f1': 0.920245398773006, 'eval_PASSPORT_f1': 0.9406224406224406, 'eval_POSTCODE_f1': 0.9815152304087478, 'eval_SECADDRESS_f1': 0.9773123909249564, 'eval_SEX_f1': 0.9750061865874784, 'eval_SOCIALNUMBER_f1': 0.9464425332290852, 'eval_STATE_f1': 0.9801093983092989, 'eval_STREET_f1': 0.9719771665801765, 'eval_TEL_f1': 0.9671641791044777, 'eval

100%|██████████| 46750/46750 [4:14:38<00:00,  3.06it/s]

{'train_runtime': 15278.6034, 'train_samples_per_second': 97.875, 'train_steps_per_second': 3.06, 'train_loss': 0.032043613471768116, 'epoch': 50.0}


TrainOutput(global_step=46750, training_loss=0.032043613471768116, metrics={'train_runtime': 15278.6034, 'train_samples_per_second': 97.875, 'train_steps_per_second': 3.06, 'total_flos': 8.79207700629273e+16, 'train_loss': 0.032043613471768116, 'epoch': 50.0})

{'eval_loss': 0.07163183391094208, 'eval_BOD_f1': 0.9608096468561584, 'eval_BUILDING_f1': 0.9815856777493606, 'eval_CARDISSUER_f1': 1.0, 'eval_CITY_f1': 0.9649595687331537, 'eval_COUNTRY_f1': 0.9744721084147495, 'eval_DATE_f1': 0.9173722627737226, 'eval_DRIVERLICENSE_f1': 0.9400864731315627, 'eval_EMAIL_f1': 0.9857495724871745, 'eval_GEOCOORD_f1': 0.9836829836829837, 'eval_GIVENNAME1_f1': 0.8495192307692307, 'eval_GIVENNAME2_f1': 0.7303058387395738, 'eval_IDCARD_f1': 0.9345971563981043, 'eval_IP_f1': 0.9868208092485549, 'eval_LASTNAME1_f1': 0.8009950248756219, 'eval_LASTNAME2_f1': 0.6885496183206107, 'eval_LASTNAME3_f1': 0.6611111111111112, 'eval_PASS_f1': 0.920245398773006, 'eval_PASSPORT_f1': 0.9406224406224406, 'eval_POSTCODE_f1': 0.9815152304087478, 'eval_SECADDRESS_f1': 0.9773123909249564, 'eval_SEX_f1': 0.9750061865874784, 'eval_SOCIALNUMBER_f1': 0.9464425332290852, 'eval_STATE_f1': 0.9801093983092989, 'eval_STREET_f1': 0.9719771665801765, 'eval_TEL_f1': 0.9671641791044777, 'eval_TIME_f1': 0.9675384413194901, 'eval_TITLE_f1': 0.9590373783922173, 'eval_USERNAME_f1': 0.9522776572668112, 'eval_precision': 0.9357036693883066, 'eval_recall': 0.9483898736158392, 'eval_f1': 0.9420040613674567, 'eval_accuracy': 0.9914942665289909, 'eval_runtime': 33.1297, 'eval_samples_per_second': 239.846, 'eval_steps_per_second': 7.516, 'epoch': 50.0}
